In [158]:
import pandas as pd
import numpy as np
import pyterrier as pt
from sklearn.ensemble import RandomForestRegressor
if not pt.started():
  pt.init()
SUBSET_SIZE = 200


In [159]:
dataset = pt.get_dataset("msmarco_passage")
#corpus = dataset.get_corpus()

In [168]:
featuresBT = pt.FeaturesBatchRetrieve(dataset.get_index("terrier_stemmed"),
                               properties={"termpipelines": ""},
                               controls = {"wmodel": "BM25"},
                               verbose=True,
                               features=["WMODEL:Tf", "WMODEL:PL2", "WMODEL:BM25", "WMODEL:DPH", "WMODEL:TF_IDF", "SAMPLE"]) % 100
bm25 = pt.BatchRetrieve(dataset.get_index("terrier_stemmed"), wmodel="BM25")

BaselineLTR = featuresBT >> pt.ltr.apply_learned_model(RandomForestRegressor(n_estimators=400))


ValueError: Must specify index variant for msmarco_passage. See http://data.terrier.org/msmarco_passage.dataset.html

In [161]:
BaselineRFR.fit(dataset.get_topics("train").sample(SUBSET_SIZE, random_state = 150), dataset.get_qrels("train"))

11:17:49.735 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


FBR(BM25 and 6 features): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [1:15:29<00:00, 22.65s/q]


In [162]:
res = pt.Experiment([BaselineLTR],
                                dataset.get_topics("test-2020").sample(SUBSET_SIZE, random_state = 150),
                                dataset.get_qrels("test-2020"), ["recip_rank", "ndcg_cut_10","map"],
                                names=["RandomForestRegressor"],
                                perquery = True)

12:33:56.937 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


FBR(BM25 and 6 features): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [1:22:44<00:00, 24.82s/q]
c:\users\pietro\appdata\local\programs\python\python38\lib\site-packages\pyterrier\pipelines.py:108: UserWarning: 146 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [164]:
allresultsLM.to_csv("./results/Random_forest.csv")

In [165]:
recip_rank = str(res[res["measure"]=="recip_rank"].mean()["value"])
ndcg_cut_10 = str(res[res["measure"]=="ndcg_cut_10"].mean()["value"])
_map = str(res[res["measure"]=="map"].mean()["value"])

C:\Users\Pietro\AppData\Local\Temp\ipykernel_69536\2117660121.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  allresultsLM[allresultsLM["measure"]=="map"].mean()


qid           inf
value    0.231119
dtype: float64

In [167]:
print(f"results\nrecip_rank: {recip_rank}\nmap: {_map}\nndcg@10: {ndcg_cut_10}")

results
recip_rank: 0.6258431823202649
map: 0.23111902041867727
ndcg@10: 0.31968094530685437
